In [0]:
# final code for svm model
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#################################

import os,sys
import csv
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from scipy.sparse import csr_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import re
from sklearn.metrics import confusion_matrix



#creating dictionary of label file
def create_train_dict(file_type):
    file_link='https://drive.google.com/open?id=1JaN_rOB059Vq9Ya3x1U4AXm47lYLn8FG'
    ## https://drive.google.com/open?id=1nQLJ-y-8XyEVQXYkR_o6HR_mnocHLsVJ   problem is there are different ids for each csv file
    # https://drive.google.com/open?id=10m8b9cX_p-oDJMwZph2oykrjZCX31F-i
    fluff, id = file_link.split('=')
    #print (id)
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('Filename.csv')  
    df3 = pd.read_csv('Filename.csv')
    train_dict={}
    list_labels=[]
    for line in df3:
        datafile=df3['Datafile']
        #print (datafile)
        label=df3['Label'] # file label           
        key=str(datafile)           #name of file
        train_dict[key]=label
        #print (train_dict[key])
        #list_labels.append(train_dict[key])
    #return (train_dict,list_labels)
    return (train_dict)
    

#printing dictionary of train.csv
def get_path(keys):
    #path = 'E:/DE/Sem2/AdvancedProject/bbdc_2019_Bewegungsdaten/'+ keys
    print (keys)
    if(keys.find("Subject02")!=-1):
      print ("yes")
      f2_link='https://drive.google.com/open?id=1r80_evl0MX34_AC7IB4v13MVps-MBEkV'
      fluff, id02 = f2_link.split('=')
      #print (id)
      downloaded = drive.CreateFile({'id':id02}) 
      #path=downloaded.GetContentFile('s2'+keys+'.csv')
      path=downloaded.GetContentFile(keys)
      print (path)
      return (path)

def label_conversion_to_numericals(train_labels):
    encoder = preprocessing.LabelEncoder()
    encoder.fit(train_labels)
    activity_labels_nums = encoder.transform(train_labels)
    return activity_labels_nums

def main(i):
    flag=0          # flag for creating array of labels
    y_label=[]
    train_labels=[]
    #y_label=np.zeros(412740)
    #train_dict,train_labels=create_train_dict(i)
    train_dict=create_train_dict(i)
    for keys in train_dict:
      label=train_dict[keys]
      train_labels.append(label)
    train_labels_arr=np.array(train_labels)
    train_labels_arr=train_labels_arr.reshape(165,1)
    #print (train_labels_arr.shape)
    #print (train_labels_arr)
    X_train_all=np.empty((0,19))
    X_test_all=np.empty((0,19))
    
# converting string labels from train.csv to integers
    if (i=='dummy_train2.csv'):
        labels_nums=label_conversion_to_numericals(train_labels_arr)
#        print (labels_nums) #array
        
    for keys in train_dict:       # keys = file name
        #print (keys)
        path=get_path(keys)
        print (path)
        f=pd.read_csv(r''+path)    # reading file
        dim=f.shape               # dim of file
        scaler = StandardScaler() # feature scaling
        if(re.match('Subject01|Subject14|Subject15',keys)==None):
            for i in range(0,dim[0]):
                y_label.append(labels_nums[flag])
                #print(labels_nums[flag])
            flag+=1
        #    print (flag)
            #print (y_label[6200])
            X_train_scaled= scaler.fit_transform(f)
            X_train_all=np.concatenate((X_train_all,X_train_scaled),axis=0)
        else:
            X_test_scaled = scaler.fit_transform(f)
            X_test_all=np.concatenate((X_test_all,X_test_scaled),axis=0)
    print(shape(y_label))
    #print ("after al concatenation",X_train_all.shape)
    # arrays formed on splitting 
    #print("x-train",X_train_all.shape,"y train", y_label.shape)
    X_train, X_test, y_train, y_test= train_test_split(X_train_all, y_label, test_size=0.3, random_state=0)  # 70% training
    num_labels=len(y_train)
    #print (num_labels)
    params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]}
    #{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]    
    svm_model = GridSearchCV(SVC(), params_grid, cv=5)
    svm_model.fit(X_train, y_train)
    #print('Best score for training data:', svm_model.best_score_,"\n") 


file=['train.csv','test.csv'] #,'dummy_test2.csv'
for i in (file):
    main(i)
    
# Predicting the Test set results
y_pred = svm_model.predict(X_test)
#Reverse factorize (converting y_pred from numbers to names
reversefactor = dict(zip(range(23),definitions))
y_test = np.vectorize(reversefactor.get)(y_test)
y_pred = np.vectorize(reversefactor.get)(y_pred)
# Making the Confusion Matrix
print(pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted']))


In [0]:
import pandas as pd

file=pd.read_csv('Subject_02_Aufnahme000.csv',encoding='utf-8')    # reading file
for line in file:
    print (line)
    break

In [0]:
a = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
b = np.array([3,4,7])
c = np.setdiff1d(a,b)
print (c)

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

X_train=open('E:/DE/Sem2/DataMining/Diogo/data_X_train.file','r')
print (X_train.read)
Y_train=open('E:/DE/Sem2/DataMining/Diogo/data_y_train.file','r')
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]},{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(X_train, Y_train)
print('Best score for training data:', svm_model.best_score_,"\n")


In [0]:
# forming y_train based on number of examples
def dup(dim,flag):
    fl=flag
    labels_nums=[2,3,4]
    for i in range(0,dim[0]):
        print (labels_nums[fl])
        y_label.append(labels_nums[fl])
    fl+=1
    return (fl,y_label)

def main():
    flag=0
    dim_list=[[10,2],[5,4]]
    y_label=[]
    for i in range(len(dim_list)):
        flag,y_label=dup(dim_list[i],flag)
        print ("flag",flag)
        print (y_label)

        

main()

In [0]:
st='Subject02_Aufnahme000.csv'
if(re.match('02',st)!=None):
  print ("ya")
